In [3]:
import numpy as np
import pandas as pd
import tarfile
import gzip
import re
import os
import datetime as dt

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

## Configure 
Configure
* number of years to import
* base directory


In [7]:
year_num = 20 # Number of past years to consider
# base_dir = "/content/drive/MyDrive/UC Davis Project 4/"
base_dir = "./"

## Import Weather Data

In [8]:
yearfiles = os.listdir(base_dir + "Resources")
yearfiles = [x for x in yearfiles if x.endswith("tar")]
yearfiles

['gsod_2014.tar',
 'gsod_2000.tar',
 'gsod_2001.tar',
 'gsod_2015.tar',
 'gsod_2003.tar',
 'gsod_2017.tar',
 'gsod_2016.tar',
 'gsod_2002.tar',
 'gsod_2006.tar',
 'gsod_2012.tar',
 'gsod_2013.tar',
 'gsod_2007.tar',
 'gsod_2011.tar',
 'gsod_2005.tar',
 'gsod_2004.tar',
 'gsod_2010.tar',
 'gsod_2009.tar',
 'gsod_2008.tar',
 'gsod_2018.tar',
 'gsod_2019.tar']

In [9]:
yearfiles.sort()
yearfiles = yearfiles[-year_num:]
yearfiles
years = [int(re.findall('\d+',yearfile)[0]) for yearfile in yearfiles]
yearfiles

['gsod_2000.tar',
 'gsod_2001.tar',
 'gsod_2002.tar',
 'gsod_2003.tar',
 'gsod_2004.tar',
 'gsod_2005.tar',
 'gsod_2006.tar',
 'gsod_2007.tar',
 'gsod_2008.tar',
 'gsod_2009.tar',
 'gsod_2010.tar',
 'gsod_2011.tar',
 'gsod_2012.tar',
 'gsod_2013.tar',
 'gsod_2014.tar',
 'gsod_2015.tar',
 'gsod_2016.tar',
 'gsod_2017.tar',
 'gsod_2018.tar',
 'gsod_2019.tar']

In [10]:
station_loc = pd.read_csv(base_dir + 'Resources/isd-history.csv')
station_loc = station_loc.replace([0.0, -999.0, -999.9],np.nan)
station_loc = station_loc[pd.notnull(station_loc['LAT']) & pd.notnull(station_loc['LON'])]
station_loc = station_loc[[int(re.findall('^\d{4}', str(end_year))[0])==max(years) for end_year in station_loc['END']]]
station_loc = station_loc[[int(re.findall('^\d{4}', str(beg_year))[0])<=min(years) for beg_year in station_loc['BEGIN']]]

In [11]:
station_loc['LBL'] = station_loc[['STATION NAME','STATE','CTRY']].apply(lambda x: x.str.cat(sep=', '), axis=1)
station_loc['ELEV_LBL'] = station_loc['ELEV(M)'].apply(lambda x: 'Elevation: '+str(x)+' m' if ~np.isnan(x) else np.nan)
station_loc['LBL'] = station_loc[['LBL','ELEV_LBL']].apply(lambda x: x.str.cat(sep='<br>'), axis=1)
# station_loc = station_loc.drop(['STATION NAME','STATE','ELEV_LBL','ICAO','BEGIN','END'], axis=1)
station_loc = station_loc.drop(['ELEV_LBL','ICAO','BEGIN','END'], axis=1)
#station_loc = station_loc.sample(stat_num)

In [12]:
# Find US Stations
station_loc.head()
us_stations = station_loc.loc[station_loc['CTRY'] == 'US', :]
us_stations_list = us_stations['USAF'].tolist()

In [13]:
df = pd.DataFrame([])
df_day = pd.DataFrame([])

def preprocess_station_file_content(content):
    headers=content.pop(0)
    headers=[headers[ind] for ind in [0,1,2,3,4,8,11,12]]
    for d in range(len(content)):
        content[d]=[content[d][ind] for ind in [0,1,2,3,5,13,17,18]]
    content=pd.DataFrame(content, columns=headers)
    content.rename(columns={'STN---': 'USAF'}, inplace=True)
    content['MAX'] = content['MAX'].apply(lambda x: re.sub("\*$","",x))
    content['MIN'] = content['MIN'].apply(lambda x: re.sub("\*$","",x))
    content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']] = content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']].apply(pd.to_numeric)
    content['YEARMODA']=pd.to_datetime(content['YEARMODA'], format='%Y%m%d', errors='ignore')
    content['YEAR']=pd.DatetimeIndex(content['YEARMODA']).year
    content['MONTH']=pd.DatetimeIndex(content['YEARMODA']).month
    content['DAY']=pd.DatetimeIndex(content['YEARMODA']).day
    return content

In [14]:
from tqdm.notebook import trange, tqdm
first = True
all_content = []
for i in trange(len(yearfiles[:-1]), desc='Years Processed'):
# for yearfile in yearfiles[:-1]:
    print(yearfiles[i])
    tar = tarfile.open(base_dir + "Resources/"+yearfiles[i], "r")
    print(len(tar.getmembers()[1:]))
    with tqdm(total=len(tar.getmembers()[1:]), desc="Processing " + yearfiles[i]) as pbar:
      i=0
      #for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
      for member in tar.getmembers()[1:]:
          name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
          usaf = name_parts[0]
          wban = int(name_parts[1])
          if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0 and (usaf in us_stations_list) and (usaf != 999999):
              i=i+1
              #if i%(stat_num//10) == 0: print(i)
              f=tar.extractfile(member)
              f=gzip.open(f, 'rb')
              content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
              # print(content)
              # print(type(content))
              if first == True:
                first = False
              else:
                content.pop(0)
              all_content = all_content + content
          pbar.update(1)

    all_content=preprocess_station_file_content(all_content)
#             df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
    df_day = df_day.append(all_content, ignore_index=True)
    all_content = all_content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
    df = df.append(all_content)
    tar.close()
    first = True
    all_content = []

Years Processed:   0%|          | 0/19 [00:00<?, ?it/s]

gsod_2000.tar
8279


Processing gsod_2000.tar:   0%|          | 0/8279 [00:00<?, ?it/s]

gsod_2001.tar
9008


Processing gsod_2001.tar:   0%|          | 0/9008 [00:00<?, ?it/s]

...

Processing gsod_2016.tar:   0%|          | 0/12076 [00:00<?, ?it/s]

gsod_2017.tar
11923


Processing gsod_2017.tar:   0%|          | 0/11923 [00:00<?, ?it/s]

gsod_2018.tar
12875


Processing gsod_2018.tar:   0%|          | 0/12875 [00:00<?, ?it/s]

In [15]:
df_day

,USAF,WBAN,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY
0,747020,23110,2000-01-03,45.4,32.5,4.9,54.0,30.2,2000,1,3
1,747020,23110,2000-01-04,45.9,32.4,3.3,53.6,37.4,2000,1,4
2,747020,23110,2000-01-05,48.2,38.2,13.2,51.8,44.6,2000,1,5
3,747020,23110,2000-01-06,39.4,32.8,3.7,51.8,26.6,2000,1,6
4,747020,23110,2000-01-07,43.4,32.2,4.3,53.6,28.4,2000,1,7
...,...,...,...,...,...,...,...,...,...,...,...
4339714,722015,12850,2018-12-27,77.3,69.4,15.8,82.9,70.0,2018,12,27
4339715,722015,12850,2018-12-28,78.9,72.2,12.2,84.9,77.0,2018,12,28
4339716,722015,12850,2018-12-29,78.5,73.5,9.4,84.9,75.0,2018,12,29
4339717,722015,12850,2018-12-30,78.6,69.9,11.2,84.0,75.0,2018,12,30


In [16]:
us_stations

,USAF,WBAN,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
14479,690150,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m"
15094,700260,27502,W POST-WILL ROGERS MEMORIAL AIRPORT,US,AK,71.283,-156.782,9.5,"W POST-WILL ROGERS MEMORIAL AIRPORT, AK, US<br..."
15098,700300,27503,WAINWRIGHT AIRPORT,US,AK,70.639,-159.995,9.1,"WAINWRIGHT AIRPORT, AK, US<br>Elevation: 9.1 m"
15116,700637,27406,DEADHORSE AIRPORT,US,AK,70.192,-148.477,18.6,"DEADHORSE AIRPORT, AK, US<br>Elevation: 18.6 m"
15117,700638,99999,FALSE PASS,US,AK,54.850,-163.417,6.0,"FALSE PASS, AK, US<br>Elevation: 6.0 m"
...,...,...,...,...,...,...,...,...,...
29435,999999,53877,ASHEVILLE 8 SSW,US,NC,35.495,-82.614,655.6,"ASHEVILLE 8 SSW, NC, US<br>Elevation: 655.6 m"
29436,999999,53878,ASHEVILLE 13 S,US,NC,35.419,-82.557,641.0,"ASHEVILLE 13 S, NC, US<br>Elevation: 641.0 m"
29595,999999,93804,SPARTANBURG,US,SC,34.917,-81.950,244.1,"SPARTANBURG, SC, US<br>Elevation: 244.1 m"
29603,999999,93816,EFFINGHAM COUNTY MEMORIAL ARPT,US,IL,39.070,-88.533,178.9,"EFFINGHAM COUNTY MEMORIAL ARPT, IL, US<br>Elev..."


In [17]:
station_loc['WBAN'] = station_loc['WBAN'].astype(float)
station_loc

,USAF,WBAN,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
11,010010,99999.0,JAN MAYEN(NOR-NAVY),NO,NaN,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
13,010014,99999.0,SORSTOKKEN,NO,NaN,59.792,5.341,48.8,"SORSTOKKEN, NO<br>Elevation: 48.8 m"
17,010020,99999.0,VERLEGENHUKEN,NO,NaN,80.050,16.250,8.0,"VERLEGENHUKEN, NO<br>Elevation: 8.0 m"
18,010030,99999.0,HORNSUND,NO,NaN,77.000,15.500,12.0,"HORNSUND, NO<br>Elevation: 12.0 m"
21,010060,99999.0,EDGEOYA,NO,NaN,78.250,22.817,14.0,"EDGEOYA, NO<br>Elevation: 14.0 m"
...,...,...,...,...,...,...,...,...,...
29435,999999,53877.0,ASHEVILLE 8 SSW,US,NC,35.495,-82.614,655.6,"ASHEVILLE 8 SSW, NC, US<br>Elevation: 655.6 m"
29436,999999,53878.0,ASHEVILLE 13 S,US,NC,35.419,-82.557,641.0,"ASHEVILLE 13 S, NC, US<br>Elevation: 641.0 m"
29595,999999,93804.0,SPARTANBURG,US,SC,34.917,-81.950,244.1,"SPARTANBURG, SC, US<br>Elevation: 244.1 m"
29603,999999,93816.0,EFFINGHAM COUNTY MEMORIAL ARPT,US,IL,39.070,-88.533,178.9,"EFFINGHAM COUNTY MEMORIAL ARPT, IL, US<br>Elev..."


## Clean and Merge Daily Dataframe

In [18]:
clean_df_day = df_day.loc[df_day['MIN'] != 9999.9]
clean_df_day = clean_df_day.loc[clean_df_day['MAX'] != 9999.9]
clean_df_day = clean_df_day.loc[clean_df_day['TEMP'] != 9999.9]
clean_df_day = clean_df_day.loc[clean_df_day["USAF"] != "999999", :]
clean_df_day.dropna(how='any')
clean_df_day

,USAF,WBAN,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY
0,747020,23110,2000-01-03,45.4,32.5,4.9,54.0,30.2,2000,1,3
1,747020,23110,2000-01-04,45.9,32.4,3.3,53.6,37.4,2000,1,4
2,747020,23110,2000-01-05,48.2,38.2,13.2,51.8,44.6,2000,1,5
3,747020,23110,2000-01-06,39.4,32.8,3.7,51.8,26.6,2000,1,6
4,747020,23110,2000-01-07,43.4,32.2,4.3,53.6,28.4,2000,1,7
...,...,...,...,...,...,...,...,...,...,...,...
4339714,722015,12850,2018-12-27,77.3,69.4,15.8,82.9,70.0,2018,12,27
4339715,722015,12850,2018-12-28,78.9,72.2,12.2,84.9,77.0,2018,12,28
4339716,722015,12850,2018-12-29,78.5,73.5,9.4,84.9,75.0,2018,12,29
4339717,722015,12850,2018-12-30,78.6,69.9,11.2,84.0,75.0,2018,12,30


### Only merge US Stations to reduce size of merge

In [19]:
# us_stations
merged_df = pd.merge(clean_df_day, us_stations, how="outer", on="USAF")
merged_df.head()

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,747020,23110.0,2000-01-03,45.4,32.5,4.9,54.0,30.2,2000.0,1.0,3.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.95,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
1,747020,23110.0,2000-01-04,45.9,32.4,3.3,53.6,37.4,2000.0,1.0,4.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.95,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
2,747020,23110.0,2000-01-05,48.2,38.2,13.2,51.8,44.6,2000.0,1.0,5.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.95,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
3,747020,23110.0,2000-01-06,39.4,32.8,3.7,51.8,26.6,2000.0,1.0,6.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.95,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
4,747020,23110.0,2000-01-07,43.4,32.2,4.3,53.6,28.4,2000.0,1.0,7.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.95,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"


In [20]:
merged_clean_df = merged_df.loc[merged_df['MIN'] != 9999.9]
merged_clean_df = merged_clean_df.loc[merged_clean_df['MAX'] != 9999.9]
merged_clean_df = merged_clean_df.loc[merged_clean_df['TEMP'] != 9999.9]
merged_clean_df = merged_clean_df.loc[merged_clean_df["USAF"] != "999999", :]
merged_clean_df.dropna(how='any')
us_merged_df = merged_clean_df.loc[merged_clean_df["CTRY"] == "US", :]
us_merged_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,747020,23110.0,2000-01-03,45.4,32.5,4.9,54.0,30.2,2000.0,1.0,3.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
1,747020,23110.0,2000-01-04,45.9,32.4,3.3,53.6,37.4,2000.0,1.0,4.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
2,747020,23110.0,2000-01-05,48.2,38.2,13.2,51.8,44.6,2000.0,1.0,5.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
3,747020,23110.0,2000-01-06,39.4,32.8,3.7,51.8,26.6,2000.0,1.0,6.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
4,747020,23110.0,2000-01-07,43.4,32.2,4.3,53.6,28.4,2000.0,1.0,7.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4301671,724856,99999.0,2018-12-29,25.2,15.7,2.8,37.4,15.8,2018.0,12.0,29.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301672,724856,99999.0,2018-12-30,30.4,22.4,3.0,44.6,23.0,2018.0,12.0,30.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301673,724856,99999.0,2018-12-31,32.1,19.8,11.0,41.0,24.8,2018.0,12.0,31.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301674,720759,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99999,RAVALLI CO,US,MT,46.251,-114.125,1110.1,"RAVALLI CO, MT, US<br>Elevation: 1110.1 m"


In [21]:
new_df=us_merged_df.dropna(how='any')
new_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,747020,23110.0,2000-01-03,45.4,32.5,4.9,54.0,30.2,2000.0,1.0,3.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
1,747020,23110.0,2000-01-04,45.9,32.4,3.3,53.6,37.4,2000.0,1.0,4.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
2,747020,23110.0,2000-01-05,48.2,38.2,13.2,51.8,44.6,2000.0,1.0,5.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
3,747020,23110.0,2000-01-06,39.4,32.8,3.7,51.8,26.6,2000.0,1.0,6.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
4,747020,23110.0,2000-01-07,43.4,32.2,4.3,53.6,28.4,2000.0,1.0,7.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4301669,724856,99999.0,2018-12-27,33.9,26.4,8.1,37.4,28.4,2018.0,12.0,27.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301670,724856,99999.0,2018-12-28,30.1,19.6,10.9,33.8,26.6,2018.0,12.0,28.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301671,724856,99999.0,2018-12-29,25.2,15.7,2.8,37.4,15.8,2018.0,12.0,29.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301672,724856,99999.0,2018-12-30,30.4,22.4,3.0,44.6,23.0,2018.0,12.0,30.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."


In [22]:
if (new_df['WBAN_x'].equals(new_df['WBAN_y'])):
    new_df = new_df.drop(['WBAN_y'], axis=1)
    new_df = new_df.rename(columns={"WBAN_x": "WBAN"})
new_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,747020,23110.0,2000-01-03,45.4,32.5,4.9,54.0,30.2,2000.0,1.0,3.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
1,747020,23110.0,2000-01-04,45.9,32.4,3.3,53.6,37.4,2000.0,1.0,4.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
2,747020,23110.0,2000-01-05,48.2,38.2,13.2,51.8,44.6,2000.0,1.0,5.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
3,747020,23110.0,2000-01-06,39.4,32.8,3.7,51.8,26.6,2000.0,1.0,6.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
4,747020,23110.0,2000-01-07,43.4,32.2,4.3,53.6,28.4,2000.0,1.0,7.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4301669,724856,99999.0,2018-12-27,33.9,26.4,8.1,37.4,28.4,2018.0,12.0,27.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301670,724856,99999.0,2018-12-28,30.1,19.6,10.9,33.8,26.6,2018.0,12.0,28.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301671,724856,99999.0,2018-12-29,25.2,15.7,2.8,37.4,15.8,2018.0,12.0,29.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301672,724856,99999.0,2018-12-30,30.4,22.4,3.0,44.6,23.0,2018.0,12.0,30.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."


##Clean and Merge Aggregate Monthly Data
Same steps as above on the aggregate data frame

In [23]:
clean_agg_df = df.loc[df['MIN'] != 9999.9]
clean_agg_df = clean_agg_df.loc[clean_agg_df['MAX'] != 9999.9]
clean_agg_df = clean_agg_df.loc[clean_agg_df['TEMP'] != 9999.9]
clean_agg_df = clean_agg_df.loc[clean_agg_df["USAF"] != "999999", :]
clean_agg_df.dropna(how='any')
clean_agg_df

,USAF,WBAN,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY
0,690150,93121,2000,1,51.20,26.60,4.30,66.20,39.9,17.0
1,690150,93121,2000,2,53.60,32.70,6.90,66.20,44.6,15.0
2,690150,93121,2000,3,61.70,29.90,6.50,73.40,46.9,16.0
3,690150,93121,2000,4,70.55,34.35,8.15,84.55,57.1,15.5
4,690150,93121,2000,5,79.00,42.50,9.70,93.90,64.4,16.0
...,...,...,...,...,...,...,...,...,...,...
8555,912850,21504,2018,8,77.60,72.30,5.20,84.90,72.0,16.0
8556,912850,21504,2018,9,78.80,71.50,5.45,86.00,71.6,15.5
8557,912850,21504,2018,10,75.50,71.20,5.00,84.00,70.0,16.0
8558,912850,21504,2018,11,74.20,68.15,5.50,82.90,68.0,15.5


In [24]:
merged_agg_df = pd.merge(clean_agg_df, us_stations, how="outer", on="USAF")
merged_agg_df
merged_clean_agg_df = merged_agg_df.loc[merged_agg_df['MIN'] != 9999.9]
merged_clean_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df['MAX'] != 9999.9]
merged_clean_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df['TEMP'] != 9999.9]
merged_clean_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df["USAF"] != "999999", :]
merged_clean_agg_df.dropna(how='any')
us_merged_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df["CTRY"] == "US", :]
us_merged_agg_df
new_agg_df=us_merged_agg_df.dropna(how='any')
if (new_agg_df['WBAN_x'].equals(new_agg_df['WBAN_y'])):
    new_agg_df = new_agg_df.drop(['WBAN_y'], axis=1)
    new_agg_df = new_agg_df.rename(columns={"WBAN_x": "WBAN"})
new_agg_df

,USAF,WBAN_x,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,690150,93121.0,2000.0,1.0,51.20,26.60,4.30,66.20,39.9,17.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m"
1,690150,93121.0,2000.0,2.0,53.60,32.70,6.90,66.20,44.6,15.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m"
2,690150,93121.0,2000.0,3.0,61.70,29.90,6.50,73.40,46.9,16.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m"
3,690150,93121.0,2000.0,4.0,70.55,34.35,8.15,84.55,57.1,15.5,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m"
4,690150,93121.0,2000.0,5.0,79.00,42.50,9.70,93.90,64.4,16.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142737,749045,99999.0,2018.0,8.0,82.30,72.70,3.40,94.30,73.9,16.0,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m"
142738,749045,99999.0,2018.0,9.0,82.10,73.55,3.60,93.95,75.0,15.5,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m"
142739,749045,99999.0,2018.0,10.0,80.00,70.00,4.60,91.60,71.6,16.0,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m"
142740,749045,99999.0,2018.0,11.0,73.45,65.65,3.65,87.45,65.3,14.5,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m"


###Additionally Add a Timestamp (1st of the Month)

In [25]:
#Set a timestamp to the beginning of the month for aggregates
# https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns-apply-a-function-o
# https://www.w3schools.com/python/python_datetime.asp
def add_aggregate_timestamp(row):
    return dt.datetime(int(row['YEAR']), int(row['MONTH']), 1)

new_agg_df['YEARMODA'] = new_agg_df.apply(lambda row: add_aggregate_timestamp(row), axis=1)
new_agg_df

<ipython-input-25-286992c37c26>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_agg_df['YEARMODA'] = new_agg_df.apply(lambda row: add_aggregate_timestamp(row), axis=1)


,USAF,WBAN_x,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL,YEARMODA
0,690150,93121.0,2000.0,1.0,51.20,26.60,4.30,66.20,39.9,17.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-01-01
1,690150,93121.0,2000.0,2.0,53.60,32.70,6.90,66.20,44.6,15.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-02-01
2,690150,93121.0,2000.0,3.0,61.70,29.90,6.50,73.40,46.9,16.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-03-01
3,690150,93121.0,2000.0,4.0,70.55,34.35,8.15,84.55,57.1,15.5,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-04-01
4,690150,93121.0,2000.0,5.0,79.00,42.50,9.70,93.90,64.4,16.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142737,749045,99999.0,2018.0,8.0,82.30,72.70,3.40,94.30,73.9,16.0,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-08-01
142738,749045,99999.0,2018.0,9.0,82.10,73.55,3.60,93.95,75.0,15.5,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-09-01
142739,749045,99999.0,2018.0,10.0,80.00,70.00,4.60,91.60,71.6,16.0,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-10-01
142740,749045,99999.0,2018.0,11.0,73.45,65.65,3.65,87.45,65.3,14.5,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-11-01


## Save to CSV

In [26]:
new_agg_df.to_csv(base_dir + "Data/weather_aggregate.csv", index=False)

In [27]:
new_df.to_csv(base_dir + "Data/weather.csv", index=False)

## Save to Mongo

In [28]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.USWeather
collection = db.weather
db.collection.insert_many(new_df.to_dict('records'))

In [29]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.USWeatherAgg
collection = db.weather
db.collection.insert_many(new_agg_df.to_dict('records'))

## Save to SQLlite

In [36]:
import sqlite3

conn = sqlite3.connect(base_dir + 'Data/weather.sqlite')
new_df.to_sql('weather', conn, if_exists='replace', index=False)
pd.read_sql('select * from weather', conn)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL
0,747020,23110.0,2000-01-03 00:00:00,45.4,32.5,4.9,54.0,30.2,2000.0,1.0,3.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
1,747020,23110.0,2000-01-04 00:00:00,45.9,32.4,3.3,53.6,37.4,2000.0,1.0,4.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
2,747020,23110.0,2000-01-05 00:00:00,48.2,38.2,13.2,51.8,44.6,2000.0,1.0,5.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
3,747020,23110.0,2000-01-06 00:00:00,39.4,32.8,3.7,51.8,26.6,2000.0,1.0,6.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
4,747020,23110.0,2000-01-07 00:00:00,43.4,32.2,4.3,53.6,28.4,2000.0,1.0,7.0,23110,NAVAL AIR STATION,US,CA,36.333,-119.950,70.7,"NAVAL AIR STATION, CA, US<br>Elevation: 70.7 m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4301669,724856,99999.0,2018-12-27 00:00:00,33.9,26.4,8.1,37.4,28.4,2018.0,12.0,27.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301670,724856,99999.0,2018-12-28 00:00:00,30.1,19.6,10.9,33.8,26.6,2018.0,12.0,28.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301671,724856,99999.0,2018-12-29 00:00:00,25.2,15.7,2.8,37.4,15.8,2018.0,12.0,29.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
4301672,724856,99999.0,2018-12-30 00:00:00,30.4,22.4,3.0,44.6,23.0,2018.0,12.0,30.0,99999,HAWTHORNE INDUSTRIAL,US,NV,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."


In [37]:
import sqlite3

conn = sqlite3.connect(base_dir + 'Data/weather_aggregate.sqlite')
new_agg_df.to_sql('weather', conn, if_exists='replace', index=False)
pd.read_sql('select * from weather', conn)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


,USAF,WBAN_x,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL,YEARMODA
0,690150,93121.0,2000.0,1.0,51.20,26.60,4.30,66.20,39.9,17.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-01-01 00:00:00
1,690150,93121.0,2000.0,2.0,53.60,32.70,6.90,66.20,44.6,15.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-02-01 00:00:00
2,690150,93121.0,2000.0,3.0,61.70,29.90,6.50,73.40,46.9,16.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-03-01 00:00:00
3,690150,93121.0,2000.0,4.0,70.55,34.35,8.15,84.55,57.1,15.5,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-04-01 00:00:00
4,690150,93121.0,2000.0,5.0,79.00,42.50,9.70,93.90,64.4,16.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-05-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142737,749045,99999.0,2018.0,8.0,82.30,72.70,3.40,94.30,73.9,16.0,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-08-01 00:00:00
142738,749045,99999.0,2018.0,9.0,82.10,73.55,3.60,93.95,75.0,15.5,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-09-01 00:00:00
142739,749045,99999.0,2018.0,10.0,80.00,70.00,4.60,91.60,71.6,16.0,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-10-01 00:00:00
142740,749045,99999.0,2018.0,11.0,73.45,65.65,3.65,87.45,65.3,14.5,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-11-01 00:00:00
